# Build LSTM model

## Read data

In [1]:
import os
from pathlib import Path

import pandas as pd


dataDirName = Path(os.environ['DATA_DIR'], 'sepsis_prediction', 'lstm_initial_trials', '02_data_matrix', 'balanced_data')
dataDirName.mkdir(exist_ok=True, parents=True)

dataDf = pd.read_csv(Path(dataDirName, 'data_1_variable.csv'))
dataDf = dataDf.sort_values(by=['PATIENT_ID', 'EPISODE_ID', 'measurement_datetime'], ascending=True)
dataDf

,PATIENT_ID,EPISODE_ID,measurement_datetime,concept_name,value_mean,Organism_FIRST_NOTED,sepsis,target
0,62567,12641659,2019-07-22,Temperature,36.619097,2019-07-26,0.0,0.0
1,62567,12641659,2019-07-23,Temperature,36.619097,2019-07-26,0.0,0.0
2,62567,12641659,2019-07-24,Temperature,36.466667,2019-07-26,0.0,0.0
3,62567,12648702,2019-07-25,Temperature,37.771429,2019-07-26,1.0,0.0
4,62567,12648702,2019-07-26,Temperature,37.000000,2019-07-26,1.0,1.0
...,...,...,...,...,...,...,...,...
25644,2645864,17644429,2023-05-04,Temperature,36.619097,2023-05-18,0.0,0.0
25645,2645864,17644429,2023-05-05,Temperature,36.200000,2023-05-18,0.0,0.0
25646,2652541,17776102,2023-06-01,Temperature,36.140000,2023-06-12,0.0,0.0
25647,2652541,17776102,2023-06-02,Temperature,36.275000,2023-06-12,0.0,0.0


In [2]:
rowsX = []
rowsY = []
for name, group in dataDf.groupby(by=['PATIENT_ID', 'EPISODE_ID']):
    low = 0
    high = 3
    while high <= group.shape[0]:
        rowsX.append([[x] for x in list(group.value_mean[low: high])])
        rowsY.append(group.target[(high - 1): high].values[0])
        high += 1
        low += 1

In [3]:
rowsX

[[[36.61909671568089], [36.61909671568089], [36.46666666666667]],
 [[37.771428571428565], [37.0], [36.66666666666666]],
 [[37.0], [36.66666666666666], [37.016666666666666]],
 [[36.66666666666666], [37.016666666666666], [36.81666666666667]],
 [[37.016666666666666], [36.81666666666667], [36.75]],
 [[36.81666666666667], [36.75], [36.72]],
 [[36.75], [36.72], [36.9]],
 [[36.72], [36.9], [36.61909671568089]],
 [[36.9], [36.61909671568089], [36.0]],
 [[36.61909671568089], [36.0], [36.6]],
 [[36.0], [36.6], [36.9]],
 [[36.6], [36.9], [36.5]],
 [[36.9], [36.5], [36.61909671568089]],
 [[36.5], [36.61909671568089], [36.61909671568089]],
 [[36.61909671568089], [36.61909671568089], [36.61909671568089]],
 [[36.61909671568089], [36.61909671568089], [36.61909671568089]],
 [[36.61909671568089], [36.61909671568089], [36.61909671568089]],
 [[36.61909671568089], [36.61909671568089], [36.61909671568089]],
 [[36.61909671568089], [36.61909671568089], [36.61909671568089]],
 [[36.61909671568089], [36.61909671

In [4]:
import numpy as np


trainX = np.array(rowsX)
testX = np.array(rowsX)
trainY = np.array(rowsY)
testY = np.array(rowsY)


In [5]:
trainX.shape, trainY.shape

((23127, 3, 1), (23127,))

### LSTM

In [ ]:
# LSTM for international airline passengers problem with time step regression framing
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


look_back = 3

model = Sequential()
model.add(LSTM(4, input_shape=(look_back, 1)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=2)
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

2025-05-13 02:20:19.627053: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-13 02:20:19.628071: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-13 02:20:19.631164: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-13 02:20:19.639603: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747102819.653937  351000 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747102819.65

Epoch 1/10


2025-05-13 02:20:21.193934: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/vmadmin/workspace/sepsis_predictor/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


23127/23127 - 23s - 981us/step - accuracy: 0.5516 - loss: 0.6882
Epoch 2/10
23127/23127 - 22s - 960us/step - accuracy: 0.5537 - loss: 0.6876
Epoch 3/10
23127/23127 - 22s - 962us/step - accuracy: 0.5537 - loss: 0.6876
Epoch 4/10
23127/23127 - 22s - 955us/step - accuracy: 0.5537 - loss: 0.6876
Epoch 5/10
23127/23127 - 22s - 940us/step - accuracy: 0.5537 - loss: 0.6875
Epoch 6/10
23127/23127 - 22s - 939us/step - accuracy: 0.5537 - loss: 0.6875
Epoch 7/10
23127/23127 - 22s - 934us/step - accuracy: 0.5537 - loss: 0.6875
Epoch 8/10
23127/23127 - 22s - 949us/step - accuracy: 0.5537 - loss: 0.6876
Epoch 9/10
23127/23127 - 22s - 952us/step - accuracy: 0.5537 - loss: 0.6875
Epoch 10/10
23127/23127 - 22s - 959us/step - accuracy: 0.5537 - loss: 0.6875
723/723 ━━━━━━━━━━━━━━━━━━━━ 1s 678us/step
723/723 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step


In [9]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score


print('accuracy_score', accuracy_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('balanced_accuracy_score', balanced_accuracy_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('precision_score', precision_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('recall_score', recall_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('precision_recall_fscore_support', precision_recall_fscore_support(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('f1_score', f1_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('roc_auc_score', roc_auc_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))

accuracy_score 0.5536818437324339
balanced_accuracy_score 0.5
precision_score 0.0
recall_score 0.0
precision_recall_fscore_support (array([0.55368184, 0.        ]), array([1., 0.]), array([0.71273517, 0.        ]), array([12805, 10322]))
f1_score 0.0
roc_auc_score 0.5


/home/vmadmin/workspace/sepsis_predictor/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/vmadmin/workspace/sepsis_predictor/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [10]:
print('roc_auc_score', roc_auc_score(trainY, trainPredict))

roc_auc_score 0.4999624318725406


## DNN

In [71]:
from tensorflow import keras


model = keras.Sequential(
    [
        keras.Input(shape=(3, )),
        keras.layers.Dense(4, activation="relu"),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 4)              │            16 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21 (84.00 B)

 Trainable params: 21 (84.00 B)

 Non-trainable params: 0 (0.00 B)

In [72]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
    keras.metrics.Accuracy(name="accuracy"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

model.fit(trainX.reshape((trainX.shape[0], 3)), trainY, epochs=10, batch_size=1, verbose=2)

Epoch 1/10


23127/23127 - 19s - 822us/step - fn: 10006.0000 - fp: 402.0000 - loss: 0.7196 - precision: 0.4401 - recall: 0.0306 - tn: 12403.0000 - tp: 316.0000
Epoch 2/10
23127/23127 - 19s - 806us/step - fn: 10067.0000 - fp: 299.0000 - loss: 0.6887 - precision: 0.4603 - recall: 0.0247 - tn: 12506.0000 - tp: 255.0000
Epoch 3/10
23127/23127 - 19s - 825us/step - fn: 10038.0000 - fp: 322.0000 - loss: 0.6887 - precision: 0.4686 - recall: 0.0275 - tn: 12483.0000 - tp: 284.0000
Epoch 4/10
23127/23127 - 19s - 825us/step - fn: 10127.0000 - fp: 215.0000 - loss: 0.6886 - precision: 0.4756 - recall: 0.0189 - tn: 12590.0000 - tp: 195.0000
Epoch 5/10
23127/23127 - 19s - 821us/step - fn: 10223.0000 - fp: 128.0000 - loss: 0.6887 - precision: 0.4361 - recall: 0.0096 - tn: 12677.0000 - tp: 99.0000
Epoch 6/10
23127/23127 - 18s - 793us/step - fn: 10304.0000 - fp: 36.0000 - loss: 0.6890 - precision: 0.3333 - recall: 0.0017 - tn: 12769.0000 - tp: 18.0000
Epoch 7/10
23127/23127 - 18s - 788us/step - fn: 10140.0000 - fp: 2

In [73]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

723/723 ━━━━━━━━━━━━━━━━━━━━ 0s 472us/step
723/723 ━━━━━━━━━━━━━━━━━━━━ 0s 437us/step


In [74]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score


print('accuracy_score', accuracy_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('balanced_accuracy_score', balanced_accuracy_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('precision_score', precision_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('recall_score', recall_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('precision_recall_fscore_support', precision_recall_fscore_support(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('f1_score', f1_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('roc_auc_score', roc_auc_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))

accuracy_score 0.5536818437324339
balanced_accuracy_score 0.5
precision_score 0.0
recall_score 0.0
precision_recall_fscore_support (array([0.55368184, 0.        ]), array([1., 0.]), array([0.71273517, 0.        ]), array([12805, 10322]))
f1_score 0.0
roc_auc_score 0.5


/home/vmadmin/workspace/sepsis_predictor/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/vmadmin/workspace/sepsis_predictor/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [75]:
print('roc_auc_score', roc_auc_score(trainY, trainPredict))

roc_auc_score 0.5


## Traditional ML

### NB Classifier model

In [54]:
from sklearn.naive_bayes import GaussianNB


gnb = GaussianNB()
y_score = gnb.fit(trainX.reshape((trainX.shape[0], 3)), trainY).predict_proba(trainX.reshape((trainX.shape[0], 3)))
y_pred = gnb.fit(trainX.reshape((trainX.shape[0], 3)), trainY).predict(trainX.reshape((trainX.shape[0], 3)))


In [59]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


print('Accuracy', accuracy_score(trainY, y_pred))
print('Balanced Accuracy', balanced_accuracy_score(trainY, y_pred))
print('F1 Score', f1_score(trainY, y_pred))
print('Precision Score', precision_score(trainY, y_pred))
print('Recall Score', recall_score(trainY, y_pred))
print('roc_auc_score', roc_auc_score(trainY, [y[1] for y in y_score]))

Accuracy 0.5538115622432654
Balanced Accuracy 0.5002768223605979
F1 Score 0.003284072249589491
Precision Score 0.5483870967741935
Recall Score 0.0016469676419298585
roc_auc_score 0.5194905646915892


### LR Classifier

In [60]:
from sklearn.linear_model import LogisticRegression


lrc = LogisticRegression(random_state=0)
y_score = lrc.fit(trainX.reshape((trainX.shape[0], 3)), trainY).predict_proba(trainX.reshape((trainX.shape[0], 3)))
y_pred = lrc.fit(trainX.reshape((trainX.shape[0], 3)), trainY).predict(trainX.reshape((trainX.shape[0], 3)))

In [61]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


print('Accuracy', accuracy_score(trainY, y_pred))
print('Balanced Accuracy', balanced_accuracy_score(trainY, y_pred))
print('F1 Score', f1_score(trainY, y_pred))
print('Precision Score', precision_score(trainY, y_pred))
print('Recall Score', recall_score(trainY, y_pred))
print('roc_auc_score', roc_auc_score(trainY, [y[1] for y in y_score]))

Accuracy 0.552730574653003
Balanced Accuracy 0.5004090125374121
F1 Score 0.025438100621820236
Precision Score 0.4623287671232877
Recall Score 0.013078860685913582
roc_auc_score 0.5337362957289151


### XGBoost Classifier

In [63]:
from xgboost import XGBClassifier

bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
# fit model
bst.fit(trainX.reshape((trainX.shape[0], 3)), trainY)
# make predictions
y_score = bst.predict_proba(trainX.reshape((trainX.shape[0], 3)))
y_pred = bst.predict(trainX.reshape((trainX.shape[0], 3)))

In [64]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


print('Accuracy', accuracy_score(trainY, y_pred))
print('Balanced Accuracy', balanced_accuracy_score(trainY, y_pred))
print('F1 Score', f1_score(trainY, y_pred))
print('Precision Score', precision_score(trainY, y_pred))
print('Recall Score', recall_score(trainY, y_pred))
print('roc_auc_score', roc_auc_score(trainY, [y[1] for y in y_score]))

Accuracy 0.5590003026765252
Balanced Accuracy 0.5191552811647686
F1 Score 0.23055450773293096
Precision Score 0.5209682918513467
Recall Score 0.1480333268746367
roc_auc_score 0.5461838068395252
